# Lap Time prediction for Formula 1 Spanish Grand Prix

This notebook implements predictive models to estimate F1 car lap times based on various factors such as tire type, weather conditions, and track state.

## Objectives
1. Load and preprocess data from FastF1 and OpenF1  
2. Perform feature engineering to enhance predictive capability  
3. Include analysis of tire degradation and pit stops  
4. Train predictive models (XGBoost and optionally a Neural Network)  
5. Evaluate performance and visualize results  


## 1. Modules Used

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
import joblib
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset




In [ ]:
# Configurations
# Plot configs
plt.style.use('ggplot')
sns.set(style="whitegrid")
# FastF1 config
fastf1.Cache.enable_cache('../f1-strategy/f1_cache')  
# Necessary dirs created if not exist
os.makedirs('../outputs/week3', exist_ok=True)
os.makedirs('../models/week3', exist_ok=True)

## 2. Model Definition

In [ ]:
# Class for Pytorch Model
class LapTimeNN(nn.Module):
    def __init__(self, input_size):
        super(LapTimeNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        return self.model(x)

## 3. Data Loading and Preparation

We will load the data from the available Parquet files:

- **laps.parquet**: Contains information about individual laps  
- **weather.parquet**: Contains weather data  
- **intervals.parquet**: Contains information about gaps and race states  
- **pitstops.parquet**: Contains information about pit stops  


In [ ]:
def load_all_data():
    """
    Load all the datasets from parquet arvhives, verifying duplicated values
    and returning them as Dataframes
    """
    # Define paths of the archives
    laps_path = "../f1-strategy/data/raw/Spain_2023_laps.parquet"
    weather_path = "../f1-strategy/data/raw/Spain_2023_weather.parquet"
    intervals_path = "../f1-strategy/data/raw/Spain_2023_openf1_intervals.parquet"
    pitstops_path = "../f1-strategy/data/raw/Spain_2023_pitstops.parquet"
    
    # Load Dataframes
    laps_df = pd.read_parquet(laps_path)
    weather_df = pd.read_parquet(weather_path)
    intervals_df = pd.read_parquet(intervals_path)
    pitstops_df = pd.read_parquet(pitstops_path)
    
    # Print info about possible duplicated values
    print("Verifying possible duplicated columns between datasets")
    
    # Compare columns 
    all_dfs = {
        'laps_df': laps_df,
        'weather_df': weather_df, 
        'intervals_df': intervals_df, 
        'pitstops_df': pitstops_df
    }
    
    for name1, df1 in all_dfs.items():
        for name2, df2 in all_dfs.items():
            if name1 >= name2:  # Avoid duplicated comparisons
                continue
                
            common_cols = set(df1.columns).intersection(set(df2.columns))
            if common_cols:
                print(f"Common columns between {name1} and {name2}: {common_cols}")
                
                # Verifying if columns have the same data
                for col in common_cols:
                    if col in df1.columns and col in df2.columns:
                        # For verifying only if they are in both datasets and have shared values
                        # For simplicity, only some values are compared 
                            value1 = df1[col].iloc[0] if len(df1) > 0 else None
                            value2 = df2[col].iloc[0] if len(df2) > 0 else None
                            print(f"  - Column '{col}': Example value in{name1} ------ {value1}, in {name2}: {value2}")
        
    
    return laps_df, weather_df, intervals_df, pitstops_df

In [ ]:
# Load and unpack de data in different dataframes
laps_df, weather_df, intervals_df, pitstops_df = load_all_data()

In [ ]:
# Show info about shape of the datafarmes

print(f"laps_df shape: {laps_df.shape} \n")

print(f"weather_df shape: {weather_df.shape} \n")

print(f"intervals_df shape: {intervals_df.shape} \n")

print(f"pitstops_df shape: {pitstops_df.shape} \n")



In [ ]:
paths = {
    "laps": "../f1-strategy/data/raw/Spain_2023_laps.parquet",
    "weather": "../f1-strategy/data/raw/Spain_2023_weather.parquet",
    "intervals": "../f1-strategy/data/raw/Spain_2023_openf1_intervals.parquet",
    "pitstops": "../f1-strategy/data/raw/Spain_2023_pitstops.parquet"
}

for name, path in paths.items():
    print(f"Columns of  {name}: {pd.read_parquet(path).columns.tolist()}")


## 4. Initial Data Exploration

* First registers of all dataframes.
* Verify that all columns are available.

In [ ]:
# Explore first registers
print("First laps_df registers:")
display(laps_df.head())

In [ ]:
print("\n First weather_df register:")
display(weather_df.head())

In [ ]:
print("\n First pitstops_df registers:")
display(pitstops_df.head())

In [ ]:
print("\n First intervals_df registers:")
display(intervals_df.head())

In [ ]:
# Verify all the columns for laps

expected_laps_columns = [
    'LapTime', 'LapNumber', 'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 
    'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 
    'Position', 'TyreLife', 'TrackStatus', 'IsAccurate', 'Compound', 'Driver'
]

print("\nAvailable columns in laps df")
for col in expected_laps_columns:
    if col in laps_df.columns:
        dtype = pitstops_df[col].dtype
        print(f"✓ {col} ----- {dtype}")
    else:
        print(f"✗ {col}")
        

In [ ]:
# Verify all weather columns
expected_weather_columns = [
    'Time', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall', 
    'TrackTemp', 'WindDirection', 'WindSpeed'
]

print("\n Available columns in weather_df:")
for col in expected_weather_columns:
    if col in weather_df.columns:
        dtype = weather_df[col].dtype
        print(f"✓ {col} ----- {dtype}")
    else:
        print(f"✗ {col}")

In [ ]:
# Verify pitstops columns
expected_pitstop_columns = [
    'Time', 'Driver', 'LapNumber', 'PitInTime', 'Compound', 'TyreLife', 'FreshTyre'
]

print("\n Available columns in pitstop_df:") 
for col in expected_pitstop_columns:
    if col in pitstops_df.columns:
        dtype = pitstops_df[col].dtype
        print(f"✓ {col} ----- {dtype}")
    else:
        print(f"✗ {col}")

## 5. Data merging and preprocessing

We need to merge all the dataframes into a single one if we want to give it to a model.

### 5.1 Join laps and weather dataframes

In [ ]:
def merge_laps_and_weather(laps_df, weather_df):
    """Merge weather and laps dataframe into a single one"""
    
    # Eliminate Time column in weather_df to avoid conflicts
    if 'Time' in weather_df.columns:
        weather_df = weather_df.drop(columns=['Time'])
    # Find a common column or another strategy if there is not a single one
    if 'LapNumber' in weather_df.columns:
        # If LapNumber present in both Dataframes, merging by this column
        merged_df = pd.merge(laps_df, weather_df, on='LapNumber', how='left')
    else:
        # If no common column, using first time measeure for all laps
        merged_df = laps_df.copy()
        for col in weather_df.columns:
            if col not in merged_df.columns:
                # Uising the first available value
                merged_df[col] = weather_df[col].iloc[0]
    
    print(f"Result: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    return merged_df

### 5.2 Calculate laps since last stop

In [ ]:
def calculate_laps_since_pitstop(df):
    """Calculate laps since last pitstop for evey driver"""
    result_df = df.copy()
    
    # Only if we have PitNextLap column
    if 'PitNextLap' in result_df.columns:
        # FOr each driver, find their stops
        for driver in result_df['Driver'].unique():
            driver_mask = result_df['Driver'] == driver
            
            # Identify laps with stop
            pit_laps = result_df.loc[driver_mask & (result_df['PitNextLap'] == 1), 'LapNumber'].values
            
            # Calculate laps since last stop
            for idx in result_df[driver_mask].index:
                lap_num = result_df.loc[idx, 'LapNumber']
                previous_pits = [p for p in pit_laps if p < lap_num]
                
                if previous_pits:
                    result_df.loc[idx, 'LapsSincePitStop'] = lap_num - max(previous_pits)
                else:
                    # If no previous pitstop (first stint), we use LapNumber
                    # Si no hay parada previa, usar LapNumber
                    result_df.loc[idx, 'LapsSincePitStop'] = lap_num
    
    return result_df

### 5.3 Process and add pitstop data

In [ ]:
def add_pitstop_data(merged_df, pitstops_df):
    """Add pitstop data"""
    essential_columns = ['Driver', 'LapNumber', 'Compound', 'FreshTyre']
    pitstops_essential = pitstops_df[essential_columns].copy()

    pitstops_essential = pitstops_essential.rename(columns={
        'Compound': 'NextCompound',
        'FreshTyre': 'FreshTyreAfterStop'
    })

    merged_result = pd.merge(
        merged_df, 
        pitstops_essential,
        on=['Driver', 'LapNumber'],
        how='left'
    )

    merged_result['PitNextLap'] = merged_result['NextCompound'].notna().astype(int)

    merged_result = calculate_laps_since_pitstop(merged_result)

    return merged_result


In [ ]:
display(intervals_df)

### 5.4 Add interval data

In [ ]:
def skip_interval_data(merged_df, intervals_df):
    """
    Función mejorada que:
    1. Extrae información de DRS del DataFrame intervals_df
    2. Calcula gaps realistas basados en tiempos acumulados
    3. Define ventanas de undercut realistas basadas en gaps entre coches
    
    Args:
        merged_df: DataFrame principal con datos de vueltas
        intervals_df: DataFrame con datos de intervalos capturados cada 4 segundos
    
    Returns:
        DataFrame con características estratégicas añadidas
    """
    print("Procesando datos de intervalos y creando características estratégicas...")
    
    # Trabajar con una copia para no modificar el original
    data = merged_df.copy()
    
    # 1. TRANSFERIR DRS DESDE INTERVALS_DF
    # Verificar si podemos extraer información de DRS desde intervals_df
    if 'drs_window' in intervals_df.columns and 'Driver' in intervals_df.columns:
        print("Extrayendo información de DRS desde intervals_df...")
        
        # Agregar datos de DRS por piloto y vuelta (si el piloto tuvo DRS en algún momento de la vuelta)
        if 'LapNumber' in intervals_df.columns:
            # Si tenemos LapNumber en intervals_df, podemos agregar directamente
            drs_by_lap = intervals_df.groupby(['Driver', 'LapNumber'])['drs_window'].max().reset_index()
            drs_by_lap = drs_by_lap.rename(columns={'drs_window': 'DRSUsed'})
            
            # Unir con el DataFrame principal
            data = pd.merge(data, drs_by_lap, on=['Driver', 'LapNumber'], how='left')
            
            # Rellenar NaN con 0
            data['DRSUsed'] = data['DRSUsed'].fillna(0).astype(int)
        else:
            # Si no tenemos LapNumber, tenemos que hacer una aproximación
            print("ADVERTENCIA: No se encontró 'LapNumber' en intervals_df, creando DRSUsed aproximado...")
            
            # Identificar líderes por vuelta
            leaders = data['Position'] == 1
            
            # Para no líderes, asignar DRS con probabilidad
            data['DRSUsed'] = 0
            non_leaders = ~leaders
            data.loc[non_leaders, 'DRSUsed'] = np.random.choice(
                [0, 1], 
                size=non_leaders.sum(), 
                p=[0.3, 0.7]  # 70% probabilidad de DRS para coches que no son líderes
            )
    else:
        print("No se encontró información de DRS en intervals_df, creando DRSUsed aproximado...")
        # Identificar líderes por vuelta
        leaders = data['Position'] == 1
        
        # Crear columna DRSUsed
        data['DRSUsed'] = 0
        non_leaders = ~leaders
        data.loc[non_leaders, 'DRSUsed'] = np.random.choice(
            [0, 1], 
            size=non_leaders.sum(), 
            p=[0.3, 0.7]
        )
    
    # 2. CALCULAR GAPS REALISTAS Y VENTANAS DE UNDERCUT
    # Verificar que tenemos las columnas necesarias
    required_cols = ['LapNumber', 'Driver', 'Position', 'LapTime']
    if all(col in data.columns for col in required_cols):
        print("Calculando gaps realistas basados en tiempos acumulados...")
        
        # Convertir LapTime a segundos si es un timedelta
        if pd.api.types.is_timedelta64_dtype(data['LapTime']):
            data['LapTime'] = data['LapTime'].dt.total_seconds()
        
        # Calcular tiempos acumulados por piloto
        data = data.sort_values(['Driver', 'LapNumber'])
        data['CumulativeTime'] = data.groupby('Driver')['LapTime'].cumsum()
        
        # Calcular gap al líder y al coche de adelante para cada vuelta
        lap_groups = []
        
        for lap_num, lap_data in data.groupby('LapNumber'):
            # Ordenar por posición
            lap_data = lap_data.sort_values('Position')
            
            # Identificar líder
            if len(lap_data) > 0:
                leader_time = lap_data.iloc[0]['CumulativeTime']
                
                # Calcular gap al líder
                lap_data['GapToLeader'] = (lap_data['CumulativeTime'] - leader_time).round(3)
                
                # Calcular gap al coche delante
                lap_data['GapToAhead'] = 0.0  # Valor default para el líder
                
                # Para cada piloto excepto el líder, calcular la diferencia con el piloto delante
                for i in range(1, len(lap_data)):
                    gap = lap_data.iloc[i]['CumulativeTime'] - lap_data.iloc[i-1]['CumulativeTime']
                    lap_data.iloc[i, lap_data.columns.get_loc('GapToAhead')] = round(gap, 3)
                
                lap_groups.append(lap_data)
        
        # Combinar todos los grupos
        if lap_groups:
            data = pd.concat(lap_groups)
        
        # 3. DEFINIR VENTANAS DE UNDERCUT
        if 'TyreAge' in data.columns:
            print("Definiendo ventanas de undercut basadas en gaps entre coches...")
            # Condiciones típicas para undercut exitoso:
            # - Gap entre 1.8 y 3.5 segundos al coche de adelante
            # - Tus neumáticos tienen al menos 8 vueltas
            # - No eres el líder (posición > 1)
            data['UndercutWindow'] = ((data['GapToAhead'] >= 1.8) & 
                                      (data['GapToAhead'] <= 3.5) & 
                                      (data['TyreAge'] >= 8) &
                                      (data['Position'] > 1)).astype(int)
        else:
            data['UndercutWindow'] = 0
            print("No se encontró 'TyreAge' para calcular ventanas de undercut precisas")
    else:
        # Si faltan columnas necesarias, usar aproximación simple
        print("Faltan columnas necesarias para calcular gaps realistas")
        missing = [col for col in required_cols if col not in data.columns]
        print(f"Columnas faltantes: {missing}")
        
        # Aproximar GapToLeader basado en posición
        data['GapToLeader'] = (data['Position'] - 1) * 2.5  # ~2.5 segundos por posición
        data.loc[data['Position'] == 1, 'GapToLeader'] = 0
        
        # Aproximar UndercutWindow
        if 'TyreAge' in data.columns:
            data['UndercutWindow'] = ((data['TyreAge'] > 10) & (data['Position'] > 1)).astype(int)
        else:
            data['UndercutWindow'] = 0
    
    # 4. CREAR OTRAS CARACTERÍSTICAS ESTRATÉGICAS
    # IsLapped - Determinar si un piloto está doblado
    if 'GapToLeader' in data.columns:
        # Si el gap al líder es mayor a 60 segundos, probablemente está doblado
        data['IsLapped'] = (data['GapToLeader'] > 60).astype(int)
    else:
        # Aproximación basada en posición
        data['IsLapped'] = (data['Position'] > 15).astype(int)
    
    # Estadísticas finales
    drs_count = data['DRSUsed'].sum()
    drs_pct = (drs_count / len(data)) * 100
    
    undercut_count = data['UndercutWindow'].sum()
    undercut_pct = (undercut_count / len(data)) * 100
    
    print(f"\nResumen de características estratégicas creadas:")
    print(f"- DRSUsed: {drs_count} activaciones ({drs_pct:.2f}%)")
    print(f"- UndercutWindow: {undercut_count} oportunidades ({undercut_pct:.2f}%)")
    print(f"- GapToLeader: Gaps calculados basados en tiempos acumulados")
    print(f"- IsLapped: Pilotos doblados identificados")
    
    print(f"\nResultado final: {data.shape[0]} filas, {data.shape[1]} columnas")
    
    return data

### 5.5 Main Function

In [ ]:
def merge_all_data_simplified(laps_df, weather_df, intervals_df, pitstops_df):
    """Función principal simplificada para la unión de todos los datos"""
    print("Dimensiones originales:")
    print(f"- laps_df: {laps_df.shape}")
    print(f"- weather_df: {weather_df.shape}")
    print(f"- intervals_df: {intervals_df.shape}")
    print(f"- pitstops_df: {pitstops_df.shape}")
    
    # Paso 1: Unir datos de vueltas y clima
    print("\nPaso 1: Uniendo datos de vueltas y clima")
    merged_df = merge_laps_and_weather(laps_df, weather_df)
    
    # Paso 2: Añadir datos de pitstops
    print("\nPaso 2: Añadiendo datos de pitstops")
    merged_df = add_pitstop_data(merged_df, pitstops_df)
    
    # Paso 3: Omitir datos de intervals pero crear características sintéticas
    print("\nPaso 3: Creando características sintéticas")
    merged_df = skip_interval_data(merged_df, intervals_df)
    
    # Verificación final
    print("\nVerificación final:")
    print(f"Dimensiones del DataFrame final: {merged_df.shape}")
    print(f"Columnas del DataFrame final: {merged_df.columns.tolist()}")
    
    # Verificar columnas estratégicas
    strategic_cols = ['DRSUsed', 'UndercutWindow', 'IsLapped', 'GapToLeader']
    for col in strategic_cols:
        if col in merged_df.columns:
            if merged_df[col].dtype == bool:
                # Convertir booleanos a enteros
                merged_df[col] = merged_df[col].astype(int)
            
            # Mostrar distribución de valores
            if col in ['DRSUsed', 'UndercutWindow', 'IsLapped']:
                value_counts = merged_df[col].value_counts()
                print(f"\nDistribución de {col}:")
                print(value_counts)
                if 1 in value_counts and 0 in value_counts:
                    pct_true = value_counts[1] / (value_counts[0] + value_counts[1]) * 100
                    print(f"Porcentaje de {col}=1: {pct_true:.1f}%")
    
    return merged_df

### 5.6 Run the entire process

In [ ]:
merged_data = merge_all_data_simplified(laps_df, weather_df, intervals_df, pitstops_df)

In [ ]:
# Verificar el resultado
print(f"DataFrame combinado: {merged_data.shape[0]} filas, {merged_data.shape[1]} columnas")
display(merged_data)

## Data Integration Approach

We merge all our different data sources (lap data, weather conditions, pit stops, and interval information) into a single comprehensive DataFrame for several key reasons:

1. **Integrated Analysis**: This approach allows us to study how various factors (weather, tire compounds, pit strategies) collectively impact lap times and race performance.

2. **ML Model Preparation**: For our predictive lap time model, we need all relevant features in a unified dataset to properly capture all variables affecting performance.

3. **Simplified Analysis Flow**: Rather than performing multiple joins each time we need to analyze relationships between different data types, we handle this complexity once.

4. **Event Tracking**: We can easily track the impact of events like pit stops across multiple laps with all data in one place.

5. **Comprehensive Visualization**: This enables us to create visualizations that simultaneously show lap time evolution, tire degradation, and changing weather conditions.

For Formula 1 analysis, where everything is interconnected (tires affect lap times, weather affects tire performance, pit strategies affect position), this integrated data approach provides the most flexible foundation for both exploratory analysis and predictive modeling.

### Next Point: exploratory data analysys (EDA) and data cleaning.

As we can see in the head of the dataframe, there are some problems that need to be solved before implementing the model. Some of the most important ones are:

* Missing values.
* Columns with the same information but in different format (eg: float and strings)

## 6. Feature Engineering y Limpieza de Datos

### 6.1 Basic Data Cleaning

In [ ]:
def clean_time_data(df):
    """
    Versión completa que limpia y transforma datos de F1 para modelado:
    - Elimina columnas innecesarias
    - Convierte columnas de tiempo a segundos
    - Transforma columnas categóricas a numéricas
    - Maneja valores faltantes
    - Elimina outliers
    
    Args:
        df: DataFrame con datos de vueltas
        
    Returns:
        DataFrame limpio y transformado listo para modelado
    """
    import numpy as np
    import pandas as pd
    import os
    
    # Trabajar con una copia para no modificar el original
    data = df.copy()
    
    # Guardar el DataFrame original antes de modificarlo (para referencia)
    os.makedirs('../f1-strategy/data/raw/processed', exist_ok=True)
    df.to_csv('../f1-strategy/data/raw/processed/original_data_backup.csv', index=False)
    
    # 1. Eliminar columnas innecesarias
    columns_to_drop = [
        'Time', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime', 
        'PitOutTime', 'LapStartTime', 'LapStartDate', 'FastF1Generated', 
        'IsAccurate', 'DeletedReason'
    ]
    existing_columns = [col for col in columns_to_drop if col in data.columns]
    
    if existing_columns:
        data = data.drop(columns=existing_columns)
        print(f"Eliminadas columnas: {', '.join(existing_columns)}")
    
    # 2. Convertir LapTime y tiempos de sector a segundos
    if 'LapTime' in data.columns and pd.api.types.is_timedelta64_dtype(data['LapTime']):
        data['LapTime'] = data['LapTime'].dt.total_seconds()
        print("Convertida LapTime a segundos.")
    
    for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']:
        if sector in data.columns and pd.api.types.is_timedelta64_dtype(data[sector]):
            data[sector] = data[sector].dt.total_seconds()
            print(f"Convertido {sector} a segundos.")
    
    # 3. Transformar PitInTime a LapToPit
    if 'PitInTime' in data.columns:
        # Crear nueva columna LapToPit basada en LapNumber donde PitInTime no es nulo
        data['LapToPit'] = 0  # Valor por defecto
        
        # Para cada fila donde PitInTime no es nulo, establecer LapToPit = LapNumber
        pit_mask = data['PitInTime'].notna()
        if pit_mask.sum() > 0:
            data.loc[pit_mask, 'LapToPit'] = data.loc[pit_mask, 'LapNumber']
            
        # Eliminar la columna PitInTime original
        data = data.drop(columns=['PitInTime'])
        print(f"Transformada PitInTime a LapToPit. Detectadas {pit_mask.sum()} entradas a pit.")
    
    # 4. Transformar Deleted a variable numérica (si existe)
    if 'Deleted' in data.columns:
        # Convertir a entero (False=0, True=1)
        data['Deleted'] = data['Deleted'].astype(int)
        print(f"Convertida Deleted a valores 0/1. Hay {data['Deleted'].sum()} vueltas eliminadas.")
    
    # 5. Transformar Team a valores numéricos
    if 'Team' in data.columns:
        # Guardar valores originales para ver exactamente qué nombres de equipos hay
        team_values = data['Team'].value_counts()
        print(f"Valores originales de Team:\n{team_values}")
        
        # Mapeo de equipos a valores numéricos incluyendo todas las variantes de nombres
        team_mapping = {
            # Equipos con el nombre exacto como aparecen en los datos
            'Alfa Romeo': 1,          # Kick Sauber
            'AlphaTauri': 2,          # Racing Bulls
            'Alpine': 3,              # Alpine
            'Aston Martin': 4,        # Aston Martin
            'Ferrari': 5,             # Ferrari
            'Haas F1 Team': 6,        # Haas
            'McLaren': 7,             # McLaren
            'Mercedes': 8,            # Mercedes
            'Red Bull Racing': 9,     # Red Bull
            'Williams': 10,           # Williams
            
            # Nombres alternativos por si acaso
            'Kick Sauber': 1,
            'Racing Bulls': 2,
            'Haas': 6,
            'Red Bull': 9,
            'RB': 2
        }
        
        # Aplicar mapeo
        data['TeamID'] = data['Team'].map(team_mapping)
        
        # Verificar si quedan equipos sin mapear
        unmapped = data[data['TeamID'].isna()]['Team'].unique()
        if len(unmapped) > 0:
            print(f"ADVERTENCIA: Equipos sin mapear: {unmapped}")
            # En caso de error, asignar valores secuenciales
            next_id = max(team_mapping.values()) + 1
            for team in unmapped:
                team_mapping[team] = next_id
                data.loc[data['Team'] == team, 'TeamID'] = next_id
                print(f"Asignado ID {next_id} a equipo desconocido: {team}")
                next_id += 1
        else:
            print("Todos los equipos mapeados correctamente.")
                
        # Eliminar columna original después de verificar mapeo
        data = data.drop(columns=['Team'])
        print(f"Transformada Team a TeamID con {len(set(team_mapping.values()))} valores únicos (1-10).")
    
    # 6. Transformar NextCompound
    if 'NextCompound' in data.columns:
        # Crear mapeo
        compound_mapping = {
            'SOFT': 1,
            'MEDIUM': 2,
            'HARD': 3,
            'INTERMEDIATE': 4,
            'WET': 5
        }
        
        # Guardar mapeo para referencia
        if not data['NextCompound'].isna().all():
            compound_values = data['NextCompound'].value_counts(dropna=False)
            print(f"Valores originales de NextCompound:\n{compound_values}")
        
        # Crear nueva columna con valores mapeados
        data['NextCompoundID'] = data['NextCompound'].map(compound_mapping)
        
        # Rellenar NaN con 0 (sin cambio de compuesto)
        data['NextCompoundID'] = data['NextCompoundID'].fillna(0).astype(int)
        
        # Eliminar columna original
        data = data.drop(columns=['NextCompound'])
        print("Transformada NextCompound a NextCompoundID (0=sin cambio, 1=soft, 2=medium, 3=hard, etc.)")
    
    # 7. Manejar FreshTyreAfterStop
    if 'FreshTyreAfterStop' in data.columns:
        # Convertir a entero (False=0, True=1)
        data['FreshTyreAfterStop'] = data['FreshTyreAfterStop'].fillna(0).astype(int)
        print("Transformada FreshTyreAfterStop a valores 0/1 (0=no fresco o sin parada)")
    
    # 8. Transformar Compound actual
    if 'Compound' in data.columns:
        # Usar el mismo mapeo que para NextCompound
        compound_mapping = {
            'SOFT': 1,
            'MEDIUM': 2,
            'HARD': 3,
            'INTERMEDIATE': 4,
            'WET': 5
        }
        
        # Guardar para referencia
        compound_values = data['Compound'].value_counts(dropna=False)
        print(f"Valores originales de Compound:\n{compound_values}")
        
        # Mapear y verificar valores faltantes
        data['CompoundID'] = data['Compound'].map(compound_mapping)
        
        # Manejar valores no mapeados
        unmapped = data[data['CompoundID'].isna()]['Compound'].unique()
        if len(unmapped) > 0:
            print(f"ADVERTENCIA: Compuestos sin mapear: {unmapped}")
            # Asignar valores para compuestos no mapeados
            next_id = max(compound_mapping.values()) + 1
            for compound in unmapped:
                compound_mapping[compound] = next_id
                data.loc[data['Compound'] == compound, 'CompoundID'] = next_id
                next_id += 1
        
        # Eliminar columna original
        data = data.drop(columns=['Compound'])
        print("Transformada Compound a CompoundID (1=soft, 2=medium, 3=hard, etc.)")
    
    # 9. Antes de eliminar outliers, guardar estos datos en un DataFrame separado
    if 'LapTime' in data.columns:
        # Identificar outliers (vueltas muy rápidas o muy lentas)
        q1 = data['LapTime'].quantile(0.05)
        q3 = data['LapTime'].quantile(0.95)
        
        # Datos que se considerarían outliers
        outlier_data = data[(data['LapTime'] < q1) | (data['LapTime'] > q3)].copy()
        
        # Añadir columna para clasificar el tipo de outlier
        outlier_data['OutlierType'] = 'Unknown'
        outlier_data.loc[outlier_data['LapTime'] < q1, 'OutlierType'] = 'VeryFast'
        outlier_data.loc[outlier_data['LapTime'] > q3, 'OutlierType'] = 'VerySlow'
        
        # Guardar para uso futuro en estrategias
        outlier_data.to_csv('../f1-strategy/data/raw/processed/exceptional_laps.csv', index=False)
        print(f"Guardados {len(outlier_data)} registros de vueltas excepcionales para análisis estratégico.")
        
        # Continuar con el filtrado para el modelo predictivo
        data = data[(data['LapTime'] >= q1) & (data['LapTime'] <= q3)]
        print(f"Filtrados outliers para el modelo predictivo. Rango válido: {q1:.2f}s - {q3:.2f}s")
    
    # 10. Guardar versión limpia para referencia
    data.to_csv('../f1-strategy/data/raw/processed/cleaned_data.csv', index=False)
    print(f"Guardado dataset limpio con {data.shape[0]} filas y {data.shape[1]} columnas.")
    
    return data

In [ ]:
# Ejecutar la limpieza de datos
cleaned_data = clean_time_data(merged_data)

# Mostrar las dimensiones antes y después
print(f"Dimensiones antes de limpieza: {merged_data.shape}")
print(f"Dimensiones después de limpieza: {cleaned_data.shape}")

display(cleaned_data)

# Data Cleaning and Transformation Strategy

## Columns Removed

We removed several columns from the dataset to improve model performance and reduce dimensionality:

1. **Time-related columns**:
   - `Time`: Redundant timestamp information that doesn't provide predictive value
   - `Sector1SessionTime`, `Sector2SessionTime`, `Sector3SessionTime`: Absolute timing information that's not relevant for lap time prediction
   - `LapStartTime`, `LapStartDate`: Absolute timing that doesn't impact lap performance
   - `PitOutTime`: 100% missing values, no usable information

2. **Quality/metadata columns**:
   - `FastF1Generated`: Metadata about data source, not a race performance factor
   - `IsAccurate`: Data quality flag that doesn't impact predictive modeling
   - `DeletedReason`: Only contained "track limits" information which is already captured in the `Deleted` flag

## Columns Transformed

We transformed several columns to improve their utility for machine learning:

1. **Time conversions**:
   - `LapTime`, `Sector1Time`, `Sector2Time`, `Sector3Time`: Converted from timedelta objects to seconds (float) for direct mathematical operations

2. **Pit stop information**:
   - `PitInTime` → `LapToPit`: Converted from timestamp to binary indicator (0 = no pit, actual lap number = pit entry) to represent when a driver entered the pits
   - `NextCompound` → `NextCompoundID`: Mapped compound names to integers (0 = no change, 1 = soft, 2 = medium, 3 = hard, etc.)
   - `FreshTyreAfterStop`: Filled NaN values with 0 (no fresh tire) and converted to integer (0/1)

3. **Categorical conversions**:
   - `Team` → `TeamID`: Mapped team names to integers (1-10) following the team order in the championship
   - `Compound` → `CompoundID`: Mapped tire compounds to integers (1 = soft, 2 = medium, 3 = hard, etc.)
   - `Deleted`: Converted boolean to integer (0/1)

## Outlier Handling

We implemented a robust outlier detection and handling strategy:

1. Identified outliers in lap times using the 5th and 95th percentiles
2. Classified outliers as "VeryFast" or "VerySlow" laps
3. Stored outliers separately for potential strategic analysis
4. Removed outliers from the training dataset to improve model quality

## Data Preservation

Throughout the cleaning process, we maintained data integrity by:

1. Working with copies of the original dataframe
2. Saving the original data before modifications
3. Documenting all transformations with clear logging
4. Saving both the exceptional laps and cleaned datasets for future reference

These transformations significantly improve the dataset's suitability for machine learning while preserving the essential racing performance information needed for accurate lap time prediction.

### 6.2 Create features related with tyres and impact on performance

In [ ]:
def create_tyre_features(df):
    """
    Crea características relacionadas con neumáticos y su impacto en el rendimiento
    
    Args:
        df: DataFrame con datos limpios
        
    Returns:
        DataFrame con nuevas características de neumáticos
    """
    data = df.copy()
    
    # 1. Edad de los neumáticos
    if 'TyreLife' in data.columns:
        data['TyreAge'] = data['TyreLife']
        print("Creada feature: TyreAge")
    
    # 2. Cambio de posición (comparado con la vuelta anterior)
    if 'Position' in data.columns and 'Driver' in data.columns:
        data['PositionChange'] = data.groupby('Driver')['Position'].diff().fillna(0)
        print("Creada feature: PositionChange")
    
    # 3. Carga de combustible (aproximación basada en la vuelta)
    if 'LapNumber' in data.columns:
        max_lap = data['LapNumber'].max()
        data['FuelLoad'] = 1 - (data['LapNumber'] / max_lap).round(4)  # Aproximación simple
        print("Creada feature: FuelLoad (aproximación)")
        
    return data

In [ ]:
# Crear características de neumáticos
cleaned_tyre_features_data = create_tyre_features(cleaned_data)

# Mostrar nuevas columnas
new_columns = set(cleaned_tyre_features_data.columns) - set(cleaned_data.columns)
print(f"Nuevas columnas creadas: {new_columns}")

display(cleaned_tyre_features_data)



In [ ]:
compound_colors = {
    1: 'red',     # SOFT
    2: 'yellow',  # MEDIUM
    3: 'gray',    # HARD
    4: 'green',   # INTERMEDIATE
    5: 'blue'     # WET
}

# Compound names for better labels in the legend
compound_names = {
    1: 'SOFT', 
    2: 'MEDIUM', 
    3: 'HARD', 
    4: 'INTERMEDIATE', 
    5: 'WET'
}

In [ ]:
# Visualizar la relación entre edad de neumáticos y tiempo por vuelta
if 'TyreAge' in cleaned_tyre_features_data.columns and 'CompoundID' in cleaned_tyre_features_data.columns:
    plt.figure(figsize=(12, 6))
    
    # Filtrar para mostrar solo compuestos principales
    for compound_id in cleaned_tyre_features_data['CompoundID'].unique():
        subset = cleaned_tyre_features_data[cleaned_tyre_features_data['CompoundID'] == compound_id]
        
        # Agrupar por edad de neumático y calcular promedio
        agg_data = subset.groupby('TyreAge')['LapTime'].mean().reset_index()
        
        # Usar el color correspondiente del diccionario actualizado
        color = compound_colors.get(compound_id, 'black')  # 'black' como color por defecto
        
        # Get readable compound name for legend
        compound_name = compound_names.get(compound_id, f'Unknown ({compound_id})')
        
        plt.plot(agg_data['TyreAge'], agg_data['LapTime'], 'o-', 
                 color=color, label=f'{compound_name} Tyre')
    
    plt.xlabel('Edad del Neumático (vueltas)')
    plt.ylabel('Tiempo por Vuelta (s)')
    plt.title('Degradación de Neumáticos: Efecto en el Tiempo por Vuelta')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('../outputs/week3/tyre_degradation_colored.png')
    plt.show()

### Tire Degradation Analysis

This plot reveals how tire age affects lap times across different compounds. 

The SOFT compound (red) generally produces slower lap times but shows inconsistent degradation patterns with several performance spikes. 

MEDIUM tires (yellow) deliver strong initial performance but gradually degrade. 

HARD tires (gray) demonstrate superior longevity, becoming the fastest option after approximately 35 laps. 

This visualization confirms the classic F1 tire performance trade-off: softer compounds offer initial speed but degrade faster, while harder compounds provide durability at the expense of initial performance.

### Create Dataframe with features related to strategies and gaps

In [ ]:
# Las características estratégicas ya existen
strategy_data = cleaned_tyre_features_data.copy()

# Redondear GapToLeader a 3 decimales
if 'GapToLeader' in strategy_data.columns:
    strategy_data['GapToLeader'] = strategy_data['GapToLeader'].round(3)
    print("GapToLeader redondeado a 3 decimales")

# Lista completa de columnas estratégicas importantes
strategic_cols = [
    # Posición y gaps
    'DRSUsed', 'GapToLeader', 'IsLapped', 'UndercutWindow',
    # Estrategia de pit stops
    'PitNextLap', 'FreshTyreAfterStop', 'LapToPit',
    # Neumáticos
    'TyreLife', 'TyreAge'
]

# Mostrar las columnas estratégicas presentes
present_cols = [col for col in strategic_cols if col in strategy_data.columns]
print(f"Características estratégicas presentes: {present_cols}")

strategy_data.head()

In [ ]:
# Si tenemos la característica DRSUsed, visualizar su impacto
if 'DRSUsed' in strategy_data.columns:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='DRSUsed', y='LapTime', data=strategy_data)
    plt.title('Impacto del DRS en Tiempos por Vuelta')
    plt.xlabel('DRS Usado (0=No, 1=Sí)')
    plt.ylabel('Tiempo por Vuelta (s)')
    plt.grid(True, alpha=0.3)
    plt.savefig('../outputs/week3/drs_impact.png')
    plt.show()

#### Analysis of DRS Impact in Barcelona
The graph reveals a surprising yet understandable outcome: the use of DRS appears to have a minimal impact on lap times at the Barcelona-Catalunya Circuit.

## Deeper Analysis of DRS Impact at the Barcelona GP

### DRS Zones
The DRS zones are located at:
1. The main straight immediately following the final corner.
2. The section between curves 9 and 10.

The **Barcelona-Catalunya Circuit** has unique characteristics that significantly influence the effectiveness of DRS:

![Barcelona-Catalunya Circuit](../outputs//week3/Spain_Circuit.jpg)

*Formula 1 official track map: [https://www.formula1.com/en/information/spain-circuit-de-barcelona-catalunya-barcelona.6F5mWJGRuYkQ1XPX48pl8]*

#### Technical Explanation
This phenomenon can be attributed to several specific features of the Barcelona circuit:

- **Limited DRS Zones:**  
  The circuit features only two relatively short DRS zones:  
  - The main straight (approximately 950m)  
  - The brief section between curves 9 and 10 (around 250m)

- **Aerodynamic Setup:**  
  Due to the numerous medium- and high-speed corners, teams generally opt for high downforce setups, which limits the performance gains from DRS.

- **Challenge of the Final Corner:**  
  The 16th corner, positioned just before the main straight, is fast and difficult to closely follow, making it hard to maintain the one-second gap required for DRS activation.

- **Circuit Balance:**  
  Only about 20% of the lap is run in sections where DRS can be effective, while the remaining 80% relies on performance in corners where DRS is not applicable.

- **Dilution of Time Gains:**  
  Although DRS can provide a gain of 0.3–0.4 seconds on specific straights, this benefit is diluted over the entire lap (typically over 1 minute and 20 seconds).

This analysis clearly illustrates why Barcelona is known as a circuit where overtaking is challenging, despite the presence of DRS, and why the results shown in the graph are not worrying. 


In [ ]:
# Extraer los tiempos por vuelta para cada piloto
lap_times = strategy_data.groupby(['LapNumber', 'Driver'])['LapTime'].mean().reset_index()

# Convertir a segundos si es necesario (en caso de que sea un objeto timedelta)
if pd.api.types.is_timedelta64_dtype(lap_times['LapTime']):
    lap_times['LapTime'] = lap_times['LapTime'].dt.total_seconds()

# Graficar los tiempos por vuelta para cada piloto
plt.figure(figsize=(14, 7))
for driver in ['VER', 'HAM', 'RUS']:
    driver_data = lap_times[lap_times['Driver'] == driver]
    if not driver_data.empty:
        plt.plot(driver_data['LapNumber'], driver_data['LapTime'], 
                 marker='o', markersize=3, linewidth=2, label=driver)

plt.title('Tiempo por Vuelta por Piloto')
plt.xlabel('Número de Vuelta')
plt.ylabel('Tiempo por Vuelta (s)')
plt.grid(True, alpha=0.3)
plt.legend()

# Añadir una línea horizontal en el tiempo de vuelta "ideal" para referencia visual
min_laptime = lap_times['LapTime'].min()
plt.axhline(y=min_laptime, color='gray', linestyle='--', alpha=0.5, 
            label=f'Mejor tiempo: {min_laptime:.2f}s')

# Ajustar rango del eje Y para mejor visualización (excluyendo valores extremos)
q1 = lap_times['LapTime'].quantile(0.05)
q3 = lap_times['LapTime'].quantile(0.95)
plt.ylim(q1 * 0.95, q3 * 1.05)

plt.tight_layout()
plt.savefig('../outputs/week3/lap_times.png')
plt.show()

In [ ]:
display(strategy_data)

### 6.4 Features of pitstops

In [ ]:
# Crear características de paradas
pitstop_data = strategy_data.copy()
# Mostrar nuevas columnas
display(pitstop_data)

In [ ]:
# Si tenemos PitNextLap, visualizar su impacto en el tiempo
if 'LapsSincePitStop' in pitstop_data.columns:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='PitNextLap', y='LapTime', data=pitstop_data)
    plt.title('Tiempos por Vuelta Antes de una Parada')
    plt.xlabel('Parada en Siguiente Vuelta (0=No, 1=Sí)')
    plt.ylabel('Tiempo por Vuelta (s)')
    plt.savefig('../outputs/week3/before_after_pit_times.png')
    plt.grid(True, alpha=0.3)
    plt.show()

## Explanation of Post-Pit Stop Lap Times

When analyzing the variable **`LapsSincePitstop`**, we often observe that the **first two laps** after exiting the pits tend to be **slower**. There are two main reasons for this:

1. **Tire Warm-Up:**  
   New tires require a lap or two to reach their optimal operating temperature. During this phase, drivers are cautious to avoid pushing too hard and overheating or flat-spotting cold tires.

2. **Tire Management:**  
   Immediately after a pit stop, drivers may also manage their tires more conservatively to prolong their life, leading to slightly slower lap times in these early laps.

After these initial laps, there is a **noticeable drop in lap times**, as the tires have reached an ideal temperature and drivers can push harder.

### Conclussion

Therefore, this graph shows that the information contained on our dataset is very well correlated with the reality of a Formula 1 race. 

---



In [ ]:
# Verificamos que exista la columna LapsSincePitStop
if 'LapsSincePitStop' in pitstop_data.columns:
    
    # Filtramos datos sólo para Verstappen
    verstappen_data = pitstop_data[pitstop_data['Driver'] == 'VER']
    
    plt.figure(figsize=(12, 5))
    
    # Limitamos a las primeras 10 vueltas después de la parada (puedes ajustar a tu gusto)
    verstappen_subset = verstappen_data[verstappen_data['LapsSincePitStop'] <= 10]
    
    # Agrupamos por vueltas desde la última parada y calculamos el promedio de lap time
    lap_groups = verstappen_subset.groupby('LapsSincePitStop')['LapTime'].mean().reset_index()
    
    # Graficamos
    plt.plot(lap_groups['LapsSincePitStop'], lap_groups['LapTime'], 'o-', linewidth=2, label='Verstappen')
    
    plt.title('Evolución del Tiempo por Vuelta de Verstappen Después de una Parada')
    plt.xlabel('Vueltas Desde Última Parada')
    plt.ylabel('Tiempo por Vuelta Promedio (s)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.savefig('../outputs/week3/after_pit_time_evo_verstappen.png')
    plt.show()


## Post-Pit Stop Performance Recovery

This chart tracks how lap times evolve after a pit stop:

1. **Slower first lap:** The first lap after the stop is significantly slower (around 5-6 seconds), likely due to pit exit speed limits and the need to bring fresh tires up to optimal temperature.  
2. **Rapid improvement:** By the second lap, lap times drop quickly as the tires start performing better.  
3. **Stabilization:** From the second or third lap onward, lap times stabilize with minor fluctuations, indicating that once the tires are “broken in,” performance remains consistent.  
4. **Undercut strategy:** This pattern supports the effectiveness of the "undercut" strategy, where despite the time lost in the pit stop, fresh tires allow for a rapid pace improvement, sometimes leading to an advantage over cars staying out longer on worn tires.  

Overall, this analysis aligns well with typical F1 race strategies.


## 6.5 Select and Prepare Final Variables

### Missing Values Analysis

This bar chart identifies data completeness issues across key features. Speed trap measurements show the highest rates of missing values, with SpeedST (straight trap) missing approximately 120,000 entries, followed by SpeedI1 (intermediate 1) and GapToLeader with around 45,000 missing values each. SpeedFL (flying lap) has fewer gaps (~15,000). 


These patterns suggest systematic data collection issues at certain track points rather than random gaps, informing our imputation strategy in the data preparation phase.

In [ ]:
# Preprocesamiento para matriz de correlación (eliminación de valores faltantes)
def prepare_correlation_features(df, max_features=10, target_col='LapTime'):
    # Hacer una copia para no modificar el original
    df_clean = df.copy()
    
    # Seleccionar solo características numéricas
    numeric_features = df_clean.select_dtypes(include=['float64', 'int64']).columns
    
    # Eliminar filas con valores faltantes en cualquier característica numérica
    df_clean = df_clean[numeric_features].dropna()
    print(f"Filas iniciales: {len(df)}, Filas después de eliminar NaN: {len(df_clean)}")
    
    # Limitar a las características más correlacionadas con LapTime
    if len(numeric_features) > max_features:
        # Asegurarse que target_col esté en el conjunto de datos
        if target_col in numeric_features:
            # Calcular correlaciones y ordenar
            corr = df_clean[numeric_features].corr()[target_col].abs().sort_values(ascending=False)
            selected_features = corr.index[:max_features]  # Top N incluye target_col
            print(f"Seleccionadas {len(selected_features)} características de {len(numeric_features)} disponibles")
            return df_clean[selected_features], selected_features
    
    # Si no hay suficientes características, devolver todas
    return df_clean[numeric_features], numeric_features


In [ ]:
# Preparar datos para correlación
clean_data, selected_features = prepare_correlation_features(final_model_data, max_features=10)

In [ ]:
# Matriz de correlación de las principales características numéricas
plt.figure(figsize=(12, 10))
sns.heatmap(clean_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación de Características Principales')
plt.tight_layout()
plt.savefig('../outputs/week3/correlation_matrix_clean.png')
plt.show()

### Feature Correlation Matrix

This correlation matrix reveals several important relationships:
1. **Lap Time Drivers**: Strong positive correlations between LapTime and FuelLoad (0.63), PitNextLap (0.62), and PitTimeLost (0.62) confirm that heavier fuel loads and approaching pit windows increase lap times.
2. **Perfect Collinearity**: LapNumber and FuelLoad are perfectly inversely correlated (-1.0), as expected since we calculated FuelLoad directly from LapNumber.
3. **Pit Stop Variables**: PitNextLap, PitTimeLost, and CompoundChange show perfect correlations, indicating redundancy.
4. **Speed Impacts**: All speed measurements negatively correlate with LapTime, confirming that higher speeds through measurement points predict lower overall lap times.

This matrix helps identify feature redundancies to remove and important relationships to preserve in our predictive model.

### Optimizing model variables

In [ ]:
# Ejecutar la preparación de datos
clean_data, selected_features = prepare_correlation_features(final_model_data, max_features=10)
print("Características seleccionadas:", selected_features.tolist())

In [ ]:
# Celda 2: Calcular y visualizar matriz de correlación de datos limpios
def calculate_correlation_matrix(clean_df):
    """
    Calcula la matriz de correlación para los datos limpios
    """
    correlation_matrix = clean_df.corr()
    
    # Visualizar matriz
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Matriz de Correlación de Características Principales')
    plt.tight_layout()
    plt.savefig('../outputs/week3/correlation_matrix_clean.png')
    plt.show()
    
    return correlation_matrix



In [ ]:
# Ejecutar el cálculo de la matriz de correlación
correlation_matrix = calculate_correlation_matrix(clean_data)

In [ ]:
# Celda 3: Analizar variables perfectamente correlacionadas
def analyze_perfect_correlations(clean_df, corr_matrix):
    # Identificar pares de variables con correlación muy alta (>0.95)
    high_corr_pairs = []
    
    # Obtener pares de alta correlación (excluyendo la diagonal)
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            col_i = corr_matrix.columns[i]
            col_j = corr_matrix.columns[j]
            corr_value = corr_matrix.iloc[i, j]
            
            if abs(corr_value) > 0.95:
                high_corr_pairs.append((col_i, col_j, corr_value))
    
    # Mostrar resultados
    if high_corr_pairs:
        print("Variables con correlación muy alta (>0.95):")
        for col_i, col_j, corr_value in high_corr_pairs:
            print(f"  - {col_i} y {col_j}: {corr_value:.2f}")
    else:
        print("No se encontraron pares de variables con correlación extremadamente alta (>0.95).")
    
    return high_corr_pairs


In [ ]:
# Ejecutar análisis de correlaciones perfectas
high_correlation_pairs = analyze_perfect_correlations(clean_data, correlation_matrix)

In [ ]:
# Celda 4: Analizar correlaciones con el tiempo por vuelta
def analyze_laptime_correlations(corr_matrix, target_col='LapTime'):
    # Extraer correlaciones con LapTime
    if target_col in corr_matrix.columns:
        laptime_corr = corr_matrix[target_col].abs().sort_values(ascending=False)
        
        # Dividir en grupos según su correlación
        high_corr = laptime_corr[laptime_corr >= 0.5]
        medium_corr = laptime_corr[(laptime_corr >= 0.25) & (laptime_corr < 0.5)]
        low_corr = laptime_corr[laptime_corr < 0.25]
        
        print(f"Correlaciones con {target_col}:")
        print("\nAlta correlación (>=0.5):")
        for feature, corr in high_corr.items():
            if feature != target_col:  # Excluir correlación consigo mismo
                print(f"  - {feature}: {corr:.2f}")
        
        print("\nCorrelación media (0.25-0.5):")
        for feature, corr in medium_corr.items():
            print(f"  - {feature}: {corr:.2f}")
        
        print("\nBaja correlación (<0.25):")
        for feature, corr in low_corr.items():
            print(f"  - {feature}: {corr:.2f}")
        
        return laptime_corr
  



In [ ]:
# Ejecutar análisis de correlaciones con LapTime
laptime_correlations = analyze_laptime_correlations(correlation_matrix)

In [ ]:
# Celda 5: Recomendar conjunto final de características
def recommend_final_features(corr_with_laptime, high_corr_pairs):
    print("\nRecomendaciones para el conjunto final de características:")
    
    # 1. Identificar características altamente predictivas
    if corr_with_laptime is not None:
        high_pred_features = corr_with_laptime[corr_with_laptime >= 0.5].index.tolist()
        if 'LapTime' in high_pred_features:
            high_pred_features.remove('LapTime')  # Eliminar la variable objetivo
        
        print("1. Características altamente predictivas a mantener:")
        for feature in high_pred_features:
            print(f"   - {feature}")
    
    # 2. Identificar redundancias a eliminar
    if high_corr_pairs:
        print("\n2. Redundancias a considerar (mantener solo una de cada par):")
        for col_i, col_j, _ in high_corr_pairs:
            # Decidir cuál mantener basado en correlación con LapTime
            if corr_with_laptime is not None:
                if corr_with_laptime.get(col_i, 0) >= corr_with_laptime.get(col_j, 0):
                    print(f"   - Mantener {col_i}, considerar eliminar {col_j}")
                else:
                    print(f"   - Mantener {col_j}, considerar eliminar {col_i}")
            else:
                print(f"   - Considerar mantener solo una de: {col_i} o {col_j}")
    
    # 3. Características con valor estratégico pero baja correlación
    if corr_with_laptime is not None:
        strategic_features = ['Position', 'TrackStatus']
        present_strategic = [f for f in strategic_features if f in corr_with_laptime.index]
        
        if present_strategic:
            print("\n3. Características con valor estratégico a mantener a pesar de baja correlación:")
            for feature in present_strategic:
                print(f"   - {feature}: {corr_with_laptime.get(feature, 0):.2f}")
    
    print("\nBalancear estos factores proporcionará un conjunto óptimo de características para el modelo final.")



In [ ]:
# Ejecutar recomendación final
recommend_final_features(laptime_correlations, high_correlation_pairs)

### Feature Selection Analysis Based on Correlation Matrix

Based on the correlation matrix analysis of our cleaned dataset, we've identified several key insights for model optimization:

1. **Highly Predictive Features**:
   - Variables with the strongest correlation to lap time should form the foundation of our predictive model.
   - This includes fuel load, speed measurements at key track sections, and tyre age.

2. **Redundancy Management**:
   - Several pairs of features show extremely high correlation (>0.95), indicating redundancy.
   - For each redundant pair, we prioritize keeping the feature with stronger correlation to lap time.
   - LapNumber and FuelLoad represent the same information in different forms - we retain both for their distinct interpretative value.

3. **Strategic Variables**:
   - Some variables like Position show lower statistical correlation but provide critical strategic insights.
   - These are retained despite lower predictive power due to their importance for race strategy decisions.

4. **Pitstop-related Features**:
   - Pitstop timing and impact variables provide crucial information for strategy modeling.
   - We created composite variables where appropriate to reduce dimensionality while preserving information.

This optimization approach balances statistical considerations with domain knowledge to create a feature set that maximizes both predictive power and strategic insight for F1 strategy decision-making.

In [ ]:
# Mostrar estadísticas descriptivas del dataset final
print("\nEstadísticas descriptivas del dataset final:")
display(clean_data.describe())

# Guardar datos procesados para modelos
# final_model_data.to_csv('data/processed/model_ready_data.csv', index=False)
print("\nDatos listos para modelado.")

In [ ]:
model_data_csv = clean_data.to_csv("../outputs/week3/model_lap_prediction.csv")

## Feature Engineering and Data Preparation

Our data preparation process follows a systematic approach broken down into specialized components:

1. **Data Cleaning**: We convert temporal data to seconds for analysis and remove outliers based on 5th and 95th percentiles to ensure our model isn't trained on anomalous lap times caused by safety cars, accidents, or data errors.

2. **Tire Performance Features**: We create features that capture tire degradation and its effect on performance, including tire age and position changes between laps.

3. **Race Strategy Features**: These features represent strategic elements like DRS usage, undercut opportunities, and gaps to the race leader that influence driving approach and lap times.

4. **Pit Stop Analysis**: We process pit stop data to calculate time lost during stops, stint length, and compound change effects that are critical for strategy modeling.

5. **Feature Selection**: Finally, we identify the most relevant variables for our model based on domain knowledge of Formula 1 racing, ensuring we include key performance drivers while avoiding redundant or noisy features.

This modular approach allows us to clearly understand each transformation and facilitates future refinements to specific aspects of data preparation.

## 7. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Resumen estadístico
print("Resumen estadístico de variables numéricas:")
display(clean_data.describe())

In [ ]:
# Tiempo por vuelta según tipo de neumático
if 'Compound' in clean_data.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Compound', y='LapTime', data=clean_data)
    plt.title('Tiempo por Vuelta según Tipo de Neumático')
    plt.xlabel('Compuesto')
    plt.ylabel('Tiempo (segundos)')
    plt.savefig('../outputs/week3/laptime_by_tyre.png')
    plt.show()

In [ ]:
# Diagrama de dispersión: Variables importantes vs LapTime
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

if 'TyreAge' in pitstop_data.columns:
    sns.scatterplot(x='TyreAge', y='LapTime', hue='Compound', data=pitstop_data, ax=axes[0])
    axes[0].set_title('LapTime vs TyreAge')
    
if 'TrackTemp' in pitstop_data.columns:
    sns.scatterplot(x='TrackTemp', y='LapTime', data=pitstop_data, ax=axes[1])
    axes[1].set_title('LapTime vs TrackTemp')
    
if 'FuelLoad' in pitstop_data.columns:
    sns.scatterplot(x='FuelLoad', y='LapTime', data=pitstop_data, ax=axes[2])
    axes[2].set_title('LapTime vs FuelLoad')

if 'Position' in pitstop_data.columns:
    sns.scatterplot(x='Position', y='LapTime', data=pitstop_data, ax=axes[3])
    axes[3].set_title('LapTime vs Position')
    
plt.tight_layout()
plt.savefig('../outputs/week3/key_variables_scatter.png')
plt.show()

## 7.1 Análisis Específico de Paradas en Boxes

In [ ]:
def visualize_pitstop_impact(data):
    """
    Visualiza el impacto de las paradas en los tiempos por vuelta
    Versión simplificada que asegura generar todos los gráficos relevantes
    """
    # Verificar disponibilidad de datos de paradas
    has_pitstop_data = ('LapsSincePitStop' in data.columns and 'PitNextLap' in data.columns)
    
    if not has_pitstop_data:
        print("No hay suficientes datos de paradas disponibles para visualización.")
        return
    
    # 1. Degradación de neumáticos por compuesto
    plt.figure(figsize=(14, 8))
    
    # Filtrar para mostrar hasta 20 vueltas después de una parada
    plot_data = data[data['LapsSincePitStop'] <= 20].copy()
    
    # Verificar si hay datos suficientes
    if len(plot_data) < 10:
        print("Datos insuficientes para analizar degradación por compuesto.")
    else:
        # Agrupar por tipo de compuesto y vueltas desde parada
        plot_data['LapsSincePitStop'] = plot_data['LapsSincePitStop'].astype(int)
        grouped = plot_data.groupby(['Compound', 'LapsSincePitStop'])['LapTime'].mean().reset_index()
        
        # Dibujar una línea por cada compuesto
        for compound in grouped['Compound'].unique():
            compound_data = grouped[grouped['Compound'] == compound]
            color = compound_colors.get(compound, 'black')
            plt.plot(compound_data['LapsSincePitStop'], 
                     compound_data['LapTime'], 
                     'o-', 
                     color=color,
                     label=f'Compuesto {compound}')
        
        plt.xlabel('Vueltas desde la parada')
        plt.ylabel('Tiempo por vuelta promedio (s)')
        plt.title('Degradación de neumáticos por compuesto')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig('../outputs/week3/tyre_degradation_by_compound.png')
        plt.show()
    
    # 2. Análisis de paradas por compuesto
    if 'PitNextLap' in data.columns:
        # Identificar vueltas previas a paradas
        pitstop_rows = data['PitNextLap'] == 1
        pitstop_count = pitstop_rows.sum()
        
        if pitstop_count > 0:
            print(f"Encontradas {pitstop_count} filas que indican paradas.")
            pitstop_data = data[pitstop_rows]
            
            # 2.1 Contar paradas por cada piloto para verificar
            pit_by_driver = pitstop_data.groupby('Driver').size()
            print("\nNúmero de paradas por piloto:")
            print(pit_by_driver)
            
            # 2.2 Gráfico de compuestos usados antes de paradas
            plt.figure(figsize=(10, 6))
            compound_counts = pitstop_data['Compound'].value_counts()
            ax = sns.barplot(x=compound_counts.index, y=compound_counts.values, 
                           palette={comp: compound_colors.get(comp, 'gray') for comp in compound_counts.index})
            
            # Añadir etiquetas
            for i, count in enumerate(compound_counts):
                ax.text(i, count/2, str(count), ha='center', va='center', fontweight='bold')
                
            plt.title('Compuestos utilizados antes de paradas')
            plt.xlabel('Compuesto')
            plt.ylabel('Número de paradas')
            plt.savefig('../outputs/week3/compounds_before_pitstop.png')
            plt.show()
        else:
            print("No se encontraron filas que indiquen paradas en boxes (PitNextLap=1).")
    else:
        print("No se encuentra la columna 'PitNextLap' para analizar paradas.")

# Ejecutar con los datos limpios
visualize_pitstop_impact(final_model_data)

## 8. Preprocesamiento para Modelado

In [ ]:
# Sección previa a "8. Preprocesamiento para Modelado"
# Finalizar la selección de variables basada en el análisis de correlación

# 1. Definir qué variables mantener según el análisis previo
variables_to_keep = [
    # Variables con alta correlación con LapTime
    'LapTime', 'FuelLoad', 'TyreAge', 'SpeedI1', 'SpeedI2',
    # Variables estratégicas
    'Position', 'Compound', 'LapsSincePitStop',
    # Información del piloto
    'Driver'
]

# 2. Verificar cuáles de estas variables están disponibles
available_vars = [var for var in variables_to_keep if var in clean_data.columns]
print(f"Variables finales seleccionadas para modelado ({len(available_vars)}/{len(variables_to_keep)}):")
for var in available_vars:
    print(f"- {var}")

# 3. Crear DataFrame final para modelado
model_ready_data = clean_data[available_vars].copy()

# 4. Guardar el dataset listo para modelado
model_ready_data.to_csv('../outputs/week3/model_ready_data.csv', index=False)
print(f"\nDatos listos para modelado guardados con {model_ready_data.shape[0]} filas y {model_ready_data.shape[1]} columnas")

# 5. Verificar valores faltantes finales
missing_values = model_ready_data.isnull().sum()
if missing_values.sum() > 0:
    print("\nValores faltantes en el dataset final:")
    print(missing_values[missing_values > 0])
else:
    print("\nNo hay valores faltantes en el dataset final.")

In [ ]:
## 8. Preprocesamiento para Modelado

def preprocess_data_for_modeling(df):
    """
    Preprocesa los datos para el modelado, dividiendo en conjuntos de entrenamiento/prueba
    y configurando los transformadores necesarios.
    
    Args:
        df: DataFrame listo para modelado
    
    Returns:
        X_train, X_test, y_train, y_test, preprocessor
    """
    # Separar características y objetivo
    X = df.drop('LapTime', axis=1)
    y = df['LapTime']
    
    # Identificar columnas categóricas y numéricas
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    print(f"Características categóricas: {cat_cols}")
    print(f"Características numéricas: {num_cols}")
    
    # Configurar transformadores para preprocesamiento
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
        ])
    
    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    print(f"Conjunto de entrenamiento: {X_train.shape[0]} muestras")
    print(f"Conjunto de prueba: {X_test.shape[0]} muestras")
    
    return X_train, X_test, y_train, y_test, preprocessor

# Preprocesar datos para modelado
X_train, X_test, y_train, y_test, preprocessor = preprocess_data_for_modeling(model_ready_data)

## 9. Entrenamiento de Modelo XGBoost

In [ ]:
def train_xgboost(X_train, X_test, y_train, y_test, preprocessor):
    """
    Entrena un modelo XGBoost
    """
    # Crear pipeline con preprocesamiento y modelo
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(objective='reg:squarederror'))
    ])
    
    # Parámetros para Grid Search
    param_grid = {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.01, 0.1],
        'regressor__max_depth': [3, 5, 7],
        'regressor__min_child_weight': [1, 3]
    }
    
    # Grid Search
    grid_search = GridSearchCV(
        model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1
    )
    
    # Entrenar modelo
    print("Entrenando modelo XGBoost con GridSearchCV (esto puede tardar varios minutos)...")
    grid_search.fit(X_train, y_train)
    
    # Mejores parámetros
    print("\nMejores parámetros XGBoost:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")
    
    # Predecir
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Evaluar
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas de evaluación del modelo XGBoost:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} segundos")
    print(f"  MAE: {mae:.4f} segundos")
    print(f"  R²: {r2:.4f}")
    
    # Analizar características importantes
    if hasattr(best_model.named_steps['regressor'], 'feature_importances_'):
        # Obtener nombres de características después del preprocesamiento
        try:
            cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
            num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
            
            # Obtener nombres de características después de one-hot encoding
            encoder = best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot']
            cat_features = encoder.get_feature_names_out(cat_cols).tolist()
            all_features = num_cols + cat_features
            
            # Crear DataFrame con importancias
            importances = best_model.named_steps['regressor'].feature_importances_
            feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importances})
            feature_importance = feature_importance.sort_values('Importance', ascending=False).head(15)
            
            # Visualizar
            plt.figure(figsize=(12, 8))
            sns.barplot(x='Importance', y='Feature', data=feature_importance)
            plt.title('Top 15 Características Más Importantes - XGBoost')
            plt.tight_layout()
            plt.savefig('outputs/week3/xgboost_feature_importance.png')
            plt.show()
        except Exception as e:
            print(f"Error al analizar importancia de características: {e}")
    
    return best_model, y_pred

# Entrenar modelo XGBoost
xgb_model, y_pred_xgb = train_xgboost(X_train, X_test, y_train, y_test, preprocessor)

## 10. Entrenamiento de Red Neuronal (Opcional)

Este paso es opcional y puede omitirse si prefieres usar solo XGBoost.

In [ ]:
def train_pytorch_nn(X_train, X_test, y_train, y_test, preprocessor):
    """
    Entrena un modelo de red neuronal con PyTorch
    """
    # Aplicar preprocesamiento
    print("Preprocesando datos para la red neuronal...")
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # Convertir a tensores
    X_train_tensor = torch.FloatTensor(X_train_processed.toarray())
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test_processed.toarray())
    y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)
    
    # Crear conjuntos de datos y cargadores
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Inicializar modelo
    print(f"Inicializando red neuronal con {X_train_processed.shape[1]} entradas...")
    input_size = X_train_processed.shape[1]
    model = LapTimeNN(input_size)
    
    # Definir criterio y optimizador
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Guardar historial de pérdidas para graficar
    losses = []
    
    # Entrenar modelo
    print("\nEntrenando red neuronal...")
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
        
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Visualizar curva de aprendizaje
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs+1), losses)
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title('Curva de Aprendizaje - Red Neuronal')
    plt.grid(True)
    plt.savefig('outputs/week3/neural_network_learning_curve.png')
    plt.show()
    
    # Evaluar modelo
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = y_pred_tensor.numpy().flatten()
        
    # Métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas de evaluación de la Red Neuronal:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} segundos")
    print(f"  MAE: {mae:.4f} segundos")
    print(f"  R²: {r2:.4f}")
    
    return model, y_pred

# Preguntar si se desea entrenar la red neuronal
train_nn = True  # Cambiar a False para omitir este paso
if train_nn:
    nn_model, y_pred_nn = train_pytorch_nn(X_train, X_test, y_train, y_test, preprocessor)
else:
    nn_model, y_pred_nn = None, None

## 11. Visualización y Comparación de Resultados

In [ ]:
def visualize_predictions(y_test, y_pred_xgb, y_pred_nn=None, max_points=1000):
    """
    Visualiza los resultados de los modelos
    """
    # Limitar número de puntos para visualización clara
    if len(y_test) > max_points:
        # Muestrear aleatoriamente para mantener la distribución
        indices = np.random.choice(len(y_test), max_points, replace=False)
        y_test_sample = y_test.iloc[indices]
        y_pred_xgb_sample = y_pred_xgb[indices]
        if y_pred_nn is not None:
            y_pred_nn_sample = y_pred_nn[indices]
    else:
        y_test_sample = y_test
        y_pred_xgb_sample = y_pred_xgb
        y_pred_nn_sample = y_pred_nn
    
    # Configurar tamaño de figura
    plt.figure(figsize=(16, 8))
    
    # Determinar número de subplots
    if y_pred_nn is not None:
        n_plots = 3
    else:
        n_plots = 2
    
    # 1. Dispersión XGBoost
    plt.subplot(1, n_plots, 1)
    plt.scatter(y_test_sample, y_pred_xgb_sample, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Tiempo real (s)')
    plt.ylabel('Tiempo predicho (s)')
    plt.title('XGBoost: Predicciones vs Reales')
    
    # 2. Residuos XGBoost
    plt.subplot(1, n_plots, 2)
    residuals_xgb = y_test_sample - y_pred_xgb_sample
    plt.scatter(y_pred_xgb_sample, residuals_xgb, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicción (s)')
    plt.ylabel('Residuos (s)')
    plt.title('XGBoost: Residuos vs Predicciones')
    
    # 3. Si hay predicciones de red neuronal
    if y_pred_nn is not None:
        plt.subplot(1, n_plots, 3)
        plt.scatter(y_test_sample, y_pred_nn_sample, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.xlabel('Tiempo real (s)')
        plt.ylabel('Tiempo predicho (s)')
        plt.title('Red Neuronal: Predicciones vs Reales')
    
    plt.tight_layout()
    plt.savefig('outputs/week3/prediction_results.png')
    plt.show()
    
    # Comparativa de errores
    if y_pred_nn is not None:
        errors_xgb = np.abs(y_test - y_pred_xgb)
        errors_nn = np.abs(y_test - y_pred_nn)
        
        plt.figure(figsize=(12, 6))
        plt.hist(errors_xgb, alpha=0.5, bins=50, label='XGBoost')
        plt.hist(errors_nn, alpha=0.5, bins=50, label='Red Neuronal')
        plt.legend()
        plt.xlabel('Error Absoluto (s)')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Errores por Modelo')
        plt.xlim(0, min(10, max(errors_xgb.max(), errors_nn.max())))
        plt.savefig('outputs/week3/error_distribution.png')
        plt.show()

# Visualizar resultados
visualize_predictions(y_test, y_pred_xgb, y_pred_nn if train_nn else None)

## 12. Guardar Modelos

In [ ]:
def save_models(xgb_model, nn_model=None):
    """
    Guarda los modelos entrenados
    """
    # Guardar modelo XGBoost
    joblib.dump(xgb_model, 'models/week3/xgboost_laptime.joblib')
    print("Modelo XGBoost guardado en 'models/week3/xgboost_laptime.joblib'")
    
    # Guardar modelo PyTorch si existe
    if nn_model is not None:
        torch.save(nn_model.state_dict(), 'models/week3/nn_laptime.pth')
        print("Modelo PyTorch guardado en 'models/week3/nn_laptime.pth'")

# Guardar modelos
save_models(xgb_model, nn_model if train_nn else None)

## 13. Prueba de Predicción con Nuevos Datos

Probemos el modelo con una situación de carrera hipotética.

In [ ]:
def test_prediction(model, available_features):
    """
    Prueba el modelo con datos hipotéticos
    """
    # Crear un escenario de ejemplo
    example_drivers = ["VER", "HAM", "LEC", "PER", "SAI"]
    compounds = ["SOFT", "MEDIUM", "HARD"]
    
    # Crear DataFrame de ejemplo
    example_data = []
    
    for driver in example_drivers:
        for compound in compounds:
            for tyre_age in [1, 10, 20]:
                # Valores predeterminados
                row = {
                    'Driver': driver,
                    'Compound': compound,
                    'TyreAge': tyre_age,
                    'LapNumber': 30,
                    'TrackTemp': 35.0,
                    'AirTemp': 25.0,
                    'Position': 5,
                    'FuelLoad': 0.5,
                    'LapsSincePitStop': tyre_age,  # Si está disponible
                    'PitNextLap': 0,  # No hay parada en la siguiente vuelta
                }
                
                # Añadir solo características disponibles
                example_row = {k: v for k, v in row.items() if k in available_features}
                example_data.append(example_row)
    
    example_df = pd.DataFrame(example_data)
    
    # Hacer predicciones
    predictions = model.predict(example_df)
    
    # Añadir predicciones al DataFrame
    example_df['PredictedLapTime'] = predictions
    
    # Visualizar resultados
    print("\nPredicciones para escenarios de ejemplo:")
    display(example_df)
    
    # Graficar por compuesto y edad de neumáticos
    if 'Compound' in example_df.columns and 'TyreAge' in example_df.columns:
        plt.figure(figsize=(12, 8))
        for driver in example_drivers[:3]:  # Limitar a 3 pilotos para claridad
            driver_data = example_df[example_df['Driver'] == driver]
            for compound in compounds:
                tyre_data = driver_data[driver_data['Compound'] == compound]
                plt.plot(tyre_data['TyreAge'], tyre_data['PredictedLapTime'], 
                         marker='o', linestyle='-', label=f"{driver} - {compound}")
                
        plt.xlabel('Edad de Neumáticos (vueltas)')
        plt.ylabel('Tiempo Predicho (s)')
        plt.title('Predicción de Tiempos por Tipo y Edad de Neumáticos')
        plt.legend()
        plt.grid(True)
        plt.savefig('outputs/week3/tyre_age_predictions.png')
        plt.show()
    
    return example_df

# Probar predicciones con el modelo XGBoost
try:
    prediction_examples = test_prediction(xgb_model, available_features)
except Exception as e:
    print(f"Error al realizar predicciones de prueba: {e}")

## 14. Conclusiones y Próximos Pasos

### Conclusiones

En este notebook, hemos:
1. Cargado y preparado datos de FastF1 y OpenF1, incluyendo información de vueltas, paradas y condiciones meteorológicas
2. Realizado feature engineering para crear características como edad de neumáticos, carga de combustible, y paradas en boxes
3. Analizado el impacto de las paradas y la degradación de neumáticos en los tiempos por vuelta
4. Entrenado un modelo XGBoost para predecir tiempos por vuelta
5. Opcionalmente, entrenado una red neuronal para comparar su rendimiento
6. Visualizado y evaluado los resultados

### Resultados Principales
- El modelo XGBoost puede predecir tiempos por vuelta con un error medio de X segundos
- Las características más importantes para la predicción son [listar las top 3-5 características]
- La degradación de neumáticos tiene un impacto significativo en los tiempos por vuelta
- Las paradas en boxes muestran patrones claros en cuanto a su timing y elección de compuesto

### Próximos Pasos (Semana 4)
1. Integrar este modelo de predicción con un sistema de decisiones basado en reglas
2. Añadir lógica para recomendar estrategias de paradas en boxes
3. Desarrollar sistema para simular undercuts/overcuts basados en las predicciones
4. Crear visualizaciones interactivas para analizar el impacto de diferentes estrategias